In [ ]:
import pandas as pd
import numpy as np
T = 1
nsteps = 48

files = [
    f'../data/export/test/100000/zerobond_1_normalize_True_test_results_sigma_0.01_dim_1_1_100000_{nsteps*T}_epochs_10.csv'
]
df_0 = pd.read_csv(
    files[0]
)[["X_0", "_delta_x_0", "dt"]]

In [ ]:
import tensorflow as tf
from utils.utils.utils import (
    ZeroBond
)
from utils.utils.utils import (
    FinanceUtils
)

delta_x = df_0._delta_x_0.values
xt = df_0.X_0.values
dt = df_0.dt.values
ct = FinanceUtils.C(
    T, 
    sigma_value = 0.01
)
nt = ZeroBond.N_tensor(
    dt,
    xt,
    ct
)
# Convert to tensors
xt = tf.convert_to_tensor(
    xt, 
    dtype = tf.float64
)
delta_x = tf.convert_to_tensor(
    delta_x,
    dtype = tf.float64
)
dt = tf.convert_to_tensor(
    dt, 
    dtype = tf.float64
)
ct = tf.Variable(
    np.float64(ct), 
    name = 'ct', 
    trainable=False
)
T = tf.Variable(
    np.float64(T), 
    name = 'T', 
    trainable=False
)

In [ ]:
h = 1e-5
y = ZeroBond.Z_tensor(
    xt, 
    dt, 
    T, 
    ct
)
y_h = ZeroBond.Z_tensor(
    xt + h, 
    dt, 
    T, 
    ct
)
grads_hand = (y_h - y) / h

In [ ]:
xt = tf.Variable(xt, name = 'xn', trainable = True)
dt = tf.Variable(dt, name = 'tn', trainable = True)
ct = tf.Variable(np.float64(ct), name = 'ct', trainable=False)
with tf.GradientTape() as tape:
    y = ZeroBond.Z_tensor(
        xt, 
        dt, 
        T, 
        ct
    )
grad_df = tape.gradient(y, {
    'xn':xt,
    'tn': dt
})
grads = grad_df['xn']
grads_dt = grad_df['tn']

In [ ]:
# Masks for prediction
idx_preds = np.array(range(0, xt.shape[0], nsteps))
np_mask_v = np.ones((xt.shape[0], 1))
np_mask_v[idx_preds] = 0
mask_v = tf.convert_to_tensor(
    np_mask_v, 
    dtype = tf.float64
)
mask_preds = tf.abs(
    tf.convert_to_tensor(
        np_mask_v,
        dtype = tf.float64
    ) - 1)

In [ ]:
# Tensor operations
batch_size = int(
    xt.shape[0] / nsteps
)
grads_reshaped = tf.reshape(grads, (batch_size, nsteps))
grads_dt_reshaped = tf.reshape(grads_dt, (batch_size, nsteps))
xt_reshaped = tf.reshape(xt, (batch_size, nsteps))
delta_x_reshaped = tf.reshape(delta_x, (batch_size, nsteps))
# Calculate the MVP
v = np.ones((batch_size, nsteps)) * 0.97039797
for i in range(1, nsteps):
    v[:, i] = (v[:, i - 1] + grads_reshaped[:, i] * delta_x_reshaped[:, i]) + grads_dt_reshaped[:, i] * T/nsteps

In [ ]:
v_real = ZeroBond.Z_tensor(
    xt, 
    dt, 
    T, 
    ct
)
v_real_reshaped = tf.reshape(
    v_real,
    (
        batch_size,
        nsteps
    )
)

In [ ]:
v_real = np.array(
    tf.reshape(v_real_reshaped[0, :], -1)
)
v_column = np.array(
    tf.reshape(v[0, :], -1)
)
dt_list = np.array(
    dt
)
df_results = pd.DataFrame(
    zip(
        v_real, 
        v_column, 
        dt_list
    ), 
    columns = [
        "v_real",
        "v_est",
        "dt"
    ]
)

In [ ]:
(df_results.v_real - df_results.v_est).abs().mean()

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
plt.figure(
    figsize = (
        10,
        5
    )
)
sns.lineplot(
    x = 'dt',
    y = 'v_real',
    data = df_results,
    color = 'red'
)
sns.lineplot(
    x = 'dt',
    y = 'v_real',
    data = df_results,
    color = 'blue'
)
plt.show()